<a href="https://colab.research.google.com/github/ajita-asthana/Node-tuts/blob/master/sosulski_kristen_dwd_class16_NYCOpenData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## We will use the data set from NYC OpenData called "New York City Leading Causes of Death".


The end point is: https://data.cityofnewyork.us/resource/jb7j-dtam.json

The URL for the data source is at: https://data.cityofnewyork.us/Health/New-York-City-Leading-Causes-of-Death/jb7j-dtam

In [ ]:
%matplotlib inline 
# It is a magic function that renders the figure in a notebook (instead of displaying the figure object reference).
import requests
import pandas as pd
import numpy as np

#### Fetching the data

We fetch the data in JSON format using the NYC OpenData API:

In [ ]:
# Data set: New York City Leading Causes of Death
url=  'https://data.cityofnewyork.us/resource/jb7j-dtam.json'
results = requests.get(url).json() 
print(len(results)) 
#note limit is defaulting to 1000. Need to add parameter for 1272 to URL using ?$limit=1272
#https://dev.socrata.com/foundry/data.cityofnewyork.us/jb7j-dtam

In [ ]:
url=  'https://data.cityofnewyork.us/resource/jb7j-dtam.json?$limit=1272'
results = requests.get(url).json()
print(len(results))
print(type(results))
print(results[1271])

Begin to look at the data.

In [ ]:
print(results[1:3]) #exclusive when using : range


In [ ]:
print(results[0]['deaths'])
print(results[0]['year'])

In [ ]:
results[0]

### Creating a DataFrame from JSON data

Let's create a pandas dataframe from the `results` 

In [ ]:
df = pd.DataFrame(results)
df

In [ ]:
len(df.index)

### Renaming Columns

We do not like some of these column names. Let's rename them.

We will use a dictionary, for specifying the existing and the new names for the columns.

In [ ]:
# This dictionary specifies as a key the existing name of the column, and as value the new name. Try this out. Change the key value pairs. 
renaming_dict = {
    'leading_cause': 'cause', 
    'sex': 'gender'
}

df.rename(columns=renaming_dict, inplace=True)
df

### Converting Data Types

In [ ]:
df.dtypes

In [ ]:
# Let's convert year using astype('int')
df['deaths'] = df['deaths'].astype('int')  
df.dtypes

In [ ]:
##ALTERNATIVE TO ABOVE FOR THE YEAR COLUMN INSTEAD...

df["year"] = pd.to_numeric(df["year"])
df.dtypes

Sometimes, during the conversion of data, the cells contain values that cannot be properly converted. We can specify how we want pandas to handle such cases. By default, it will raise an exception, and will not allow us to convert the data to a new data type.

In [ ]:
# This one will cause an error, as the "deaths" column contains non-numeric values.
df["deaths"] = pd.to_numeric(df["deaths"])

In [ ]:
df["deaths"].iloc[188]

In [ ]:

df.iloc[188:190]

We can pass the `errors` command to specify what should happen. From the [documentation of to_numeric](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_numeric.html), we get:
* If ‘raise’, then invalid parsing will raise an exception
* If ‘coerce’, then invalid parsing will be set as NaN
* If ‘ignore’, then invalid parsing will return the input

In [ ]:
df["deaths"] = pd.to_numeric(df["deaths"], errors='coerce')
df["death_rate"] = pd.to_numeric(df["death_rate"], errors='coerce')
df["age_adjusted_death_rate"] = pd.to_numeric(df["age_adjusted_death_rate"], errors='coerce')
df.dtypes

In [ ]:
df

We will also mark the other values as Categorical.

In [ ]:
df["gender"] = pd.Categorical(df["gender"])
df["race_ethnicity"] = pd.Categorical(df["race_ethnicity"])
df["cause"] = pd.Categorical(df["cause"])
df.dtypes

### Exploratory Data Analysis

In [ ]:
df["gender"].value_counts()

In [ ]:
# Let's be consistent with the gender nominal values
df['gender'].replace({'Female': 'F', 'Male': 'M'}, inplace=True)
df['gender'].value_counts()

In [ ]:
df['cause'].value_counts()

In [ ]:
df.groupby(by='gender').size()

In [ ]:
df['gender'].nunique()

### Pivot Tables

Let's create a pivot table now. We are going to put the "leading cause" as the row, with gender and race as columns. For the cell values we will use the number of deaths, and we are going to sum (`np.sum`) the values.

_Note: You will also find the `pivot` and `crosstab` functions in Pandas. The `pivot_table` function is typically a more general version of both._

In [ ]:
pivot = pd.pivot_table(df, 
                       values='deaths', 
                       index=['cause'], # rows
                       columns=['gender', 'race_ethnicity'], # columns
                       aggfunc=np.mean).copy() # aggregation function
pivot

In [ ]:
#Check the data type of pivot. 

type(pivot)

And we can easily transpose the dataframe

In [ ]:
pivot.transpose()
# alternatively you could use pivot.T

In [ ]:
pivot.transpose()['Influenza (Flu) and Pneumonia (J09-J18)'].sort_values().plot(kind='barh', title='Leading cause of death: Influenza and Pneumonia', color='#999999');

In [ ]:
pivot.transpose()['Diseases of Heart (I00-I09, I11, I13, I20-I51)'].sort_values().plot(kind='barh', title='Leading cause of death: Diseases of the Heart', color='#932211');